## 准备数据

In [16]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [17]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [18]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        # 输入层到隐藏层的权重，输入为 28*28，隐藏层神经元 256 个
        self.W1 = tf.Variable(tf.random.normal([28 * 28, 256], stddev=0.1))
        # 隐藏层的偏置
        self.b1 = tf.Variable(tf.zeros([256]))
        # 隐藏层到输出层的权重，输出为 10 个类别
        self.W2 = tf.Variable(tf.random.normal([256, 10], stddev=0.1))
        # 输出层的偏置
        self.b2 = tf.Variable(tf.zeros([10]))
        
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        # 将输入的二维图像数据展平为一维向量
        x = tf.reshape(x, [-1, 28 * 28])
        # 计算隐藏层的输出，使用 ReLU 激活函数
        h = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)
        # 计算输出层的未归一化 logits
        logits = tf.matmul(h, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [19]:
# @tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

# @tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

# @tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

# @tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [20]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.8820436 ; accuracy 0.10885
epoch 1 : loss 2.781041 ; accuracy 0.11125
epoch 2 : loss 2.698146 ; accuracy 0.11331667
epoch 3 : loss 2.628307 ; accuracy 0.115483336
epoch 4 : loss 2.5683672 ; accuracy 0.11815
epoch 5 : loss 2.516188 ; accuracy 0.121183336
epoch 6 : loss 2.4702358 ; accuracy 0.12408333
epoch 7 : loss 2.4293573 ; accuracy 0.12723333
epoch 8 : loss 2.3926477 ; accuracy 0.1312
epoch 9 : loss 2.3593993 ; accuracy 0.13573334
epoch 10 : loss 2.3290358 ; accuracy 0.14111666
epoch 11 : loss 2.3010864 ; accuracy 0.14753333
epoch 12 : loss 2.2751672 ; accuracy 0.15496667
epoch 13 : loss 2.2509632 ; accuracy 0.16465
epoch 14 : loss 2.2282104 ; accuracy 0.17393333
epoch 15 : loss 2.2067008 ; accuracy 0.18368334
epoch 16 : loss 2.1862552 ; accuracy 0.19278333
epoch 17 : loss 2.1667213 ; accuracy 0.20328334
epoch 18 : loss 2.1479757 ; accuracy 0.21221666
epoch 19 : loss 2.1299129 ; accuracy 0.22121666
epoch 20 : loss 2.1124482 ; accuracy 0.23098333
epoch 21 : loss 2.09